In [1]:
import sqlite3 as sql

import numpy as np
import pandas as pd
from pandas import cut, read_sql
import pickle as pkl
from random import choices
from sklearn import metrics
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy.optimize as optimize

In [2]:
indir = "/groups/icecube/petersen/GraphNetDatabaseRepository/osc_next_database_Peter_and_Morten/merged_database/osc_next_level3_v2.00_genie_muongun_noise_120000_140000_160000_130000_888003.db"
indir_retro = "/groups/icecube/petersen/GraphNetDatabaseRepository/osc_next_database_Peter_and_Morten/merged_database/osc_next_level3_v2.00_genie_muongun_noise_120000_140000_160000_130000_888003_retro.db"

#Load in truth data
with sql.connect(indir) as con:
    query = """
    SELECT
        event_no, RunID, SubrunID, EventID, SubEventID
    FROM 
        truth
    """
    truth = read_sql(query,con)

#Load in truth data
with sql.connect(indir_retro) as con:
    query = """
    SELECT
        event_no, RunID, SubrunID, EventID, SubEventID
    FROM 
        truth
    """
    truth_retro = read_sql(query,con)

with sql.connect(indir_retro) as con:
    query = """
    SELECT
        event_no, osc_weight
    FROM 
        retro
    """
    retro = read_sql(query,con)

In [4]:
truth.rename(columns = {'event_no':'old_event_no'}, inplace = True)

merged_df = pd.merge(truth, truth_retro, on=['RunID', 'SubrunID', 'EventID', 'SubEventID'])

#merged_df[merged_df['_merge'] == 'both']

In [10]:
print(merged_df.head(10))
print(truth[truth['old_event_no']==4])
print(truth_retro[truth_retro['event_no']==55418459])

   old_event_no     RunID  SubrunID  EventID  SubEventID  event_no
0             0  888003.0    5450.0      0.0         0.0  55418455
1             1  888003.0    5450.0      1.0         0.0  55418456
2             2  888003.0    5450.0      3.0         0.0  55418457
3             3  888003.0    5450.0      5.0         0.0  55418458
4             4  888003.0    5450.0     11.0         0.0  55418459
5             5  888003.0    5450.0     20.0         0.0  55418460
6             6  888003.0    5450.0     22.0         0.0  55418461
7             7  888003.0    5450.0     23.0         0.0  55418462
8             8  888003.0    5450.0     26.0         0.0  55418464
9             9  888003.0    5450.0     28.0         0.0  55418465
   old_event_no     RunID  SubrunID  EventID  SubEventID
4             4  888003.0    5450.0     11.0         0.0
          event_no     RunID  SubrunID  EventID  SubEventID
55418459  55418459  888003.0    5450.0     11.0         0.0


In [11]:
final_merged_df = pd.merge(merged_df, retro, on=['event_no'])

In [14]:
print(final_merged_df.head(10))
print(retro['osc_weight'][retro['event_no']==55418458	])

   old_event_no     RunID  SubrunID  EventID  SubEventID  event_no  osc_weight
0             0  888003.0    5450.0      0.0         0.0  55418455    0.003054
1             1  888003.0    5450.0      1.0         0.0  55418456    0.003054
2             2  888003.0    5450.0      3.0         0.0  55418457    0.003054
3             3  888003.0    5450.0      5.0         0.0  55418458    0.003054
4             4  888003.0    5450.0     11.0         0.0  55418459    0.003054
5             5  888003.0    5450.0     20.0         0.0  55418460    0.003054
6             6  888003.0    5450.0     22.0         0.0  55418461    0.003054
7             7  888003.0    5450.0     23.0         0.0  55418462    0.003054
8             8  888003.0    5450.0     26.0         0.0  55418464    0.003054
9             9  888003.0    5450.0     28.0         0.0  55418465    0.003054
55418458    0.003054
Name: osc_weight, dtype: float64


In [16]:
output_location = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_MP_lvl3/inference/event_selections/osc_weights.csv"
final_merged_df.to_csv(output_location,index=False,columns=('old_event_no','osc_weight'))